In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import re

In [2]:
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_rows',500)

In [3]:
play = pd.read_csv("play_by_play.csv")

In [4]:
play = play.drop(['eventnum', 'eventmsgactiontype', 'period',
       'wctimestring', 'pctimestring', 'neutraldescription', 'person1type',
       'player1_team_id', 'player1_team_city',
       'player1_team_nickname', 'player1_team_abbreviation', 'person2type',
       'player2_team_id', 'player2_team_city',
       'player2_team_nickname', 'player2_team_abbreviation', 'person3type',
       'player3_team_id', 'player3_team_city',
       'player3_team_nickname', 'player3_team_abbreviation',
       'video_available_flag'], axis = 1)

In [5]:
player = pd.DataFrame(data = {'person_id' : [], 'name' : [], 'field_goals_made' : [], 'field_goals_ast' : [], 'field_goals_missed' : [], 
                              'times_ball_lost' : [], 'times_ball_stolen' : [], 'fouls' : [], 'game_id' : [], 'games_played' : [], 
                              'field_goals_blocked' : [], 'free_throws_made' : [], 'free_throws_missed' : [], 'rebounds' : [], 
                              "points" : []})

In [6]:
game_id = 0
count = 0
game_id_index = 0
eventmsgtype = 1
homedescription = 2
visitordescription = 3
player1_id = 6
player1_name = 7
player2_id = 8
player2_name = 9
player3_id = 10
player3_name = 11
print("starting...")
for row in play.values:
    data = {'person_id' : 0, 'name' : 0, 'field_goals_made' : 0, 'field_goals_ast' : 0, 'field_goals_missed' : 0, 
            'times_ball_lost' : 0, 'times_ball_stolen' : 0, 'fouls' : 0, 'game_id' : 0, 'games_played' : 0, 'field_goals_blocked' : 0, 
            'free_throws_made' : 0, 'free_throws_missed' : 0, 'rebounds' : 0}
    if(
        (row[eventmsgtype] == 8) | (row[eventmsgtype] == 9) | (row[eventmsgtype] == 10) | 
        (row[eventmsgtype] == 11) | (row[eventmsgtype] == 12) | (row[eventmsgtype] == 13) |
        (pd.isna(row[player1_name]))
       ):
        continue
    
    elif row[eventmsgtype] == 1:
        if not player.loc[(player["name"] == row[player1_name]), "name"].empty:
            player.loc[player["name"] == row[player1_name], "field_goals_made"] += 1
        else:
            data["person_id"] = row[player1_id]
            data["name"] = row[player1_name]
            data["field_goals_made"] = 1
            player = player.append(data, ignore_index=True)

        if not pd.isna(row[player2_name]):
            if not player.loc[(player["name"] == row[player2_name]), "name"].empty:
                player.loc[player["name"] == row[player2_name], 'field_goals_ast'] += 1
            else:
                data["person_id"] = row[player2_id]
                data["name"] = row[player2_name]
                data["field_goals_ast"] = 1
                player = player.append(data, ignore_index=True)
                
    elif row[eventmsgtype] == 2:
        if not player.loc[(player["name"] == row[player1_name]), "name"].empty:
            player.loc[player["name"] == row[player1_name], "field_goals_missed"] += 1
        else:
            data["person_id"] = row[player1_id]
            data["name"] = row[player1_name]
            data["field_goals_missed"] = 1
            player = player.append(data, ignore_index=True)
        if not pd.isna(row[player3_name]):
            if not player.loc[(player["name"] == row[player3_name]), "name"].empty:
                player.loc[player["name"] == row[player3_name], "field_goals_blocked"] += 1
            else:
                data["person_id"] = row[player3_id]
                data["name"] = row[player3_name]
                data["field_goals_blocked"] = 1
                player = player.append(data, ignore_index=True)
        
    elif row[eventmsgtype] == 3:
        if not pd.isna(row[homedescription]):
            if "MISS" in row[homedescription]:
                if not player.loc[(player["person_id"] == row[player1_id]), "person_id"].empty:
                    player.loc[(player["person_id"] == row[player1_id]), "free_throws_missed"] += 1
                else:
                    data["person_id"] = row[player1_id]
                    data["name"] = row[player1_name]
                    player.loc[(player["person_id"] == row[player1_id]), "free_throws_missed"] = 1
                    player = player.append(data, ignore_index=True)
            else:
                if not player.loc[(player["person_id"] == row[player1_id]), "person_id"].empty:
                    player.loc[(player["person_id"] == row[player1_id]), "free_throws_made"] += 1
                else:
                    data["person_id"] = row[player1_id]
                    data["name"] = row[player1_name]
                    player.loc[(player["person_id"] == row[player1_id]), "free_throws_made"] = 1
                    player = player.append(data, ignore_index=True)
        else:
            if "MISS" in row[visitordescription]:
                if not player.loc[(player["person_id"] == row[player1_id]), "person_id"].empty:
                    player.loc[(player["person_id"] == row[player1_id]), "free_throws_missed"] += 1
                else:
                    data["person_id"] = row[player1_id]
                    data["name"] = row[player1_name]
                    player.loc[(player["person_id"] == row[player1_id]), "free_throws_missed"] = 1
                    player = player.append(data, ignore_index=True)
            else:
                if not player.loc[(player["person_id"] == row[player1_id]), "person_id"].empty:
                    player.loc[(player["person_id"] == row[player1_id]), "free_throws_made"] += 1
                else:
                    data["person_id"] = row[player1_id]
                    data["name"] = row[player1_name]
                    player.loc[(player["person_id"] == row[player1_id]), "free_throws_made"] = 1
                    player = player.append(data, ignore_index=True)
        
    elif row[eventmsgtype] == 4:
        if not player.loc[(player["person_id"] == row[player1_id]), "person_id"].empty:
            player.loc[(player["person_id"] == row[player1_id]), "rebounds"] += 1
        else:
            data["name"] = row[player1_name]
            data["person_id"] = row[player1_id]
            data["rebounds"] = 1
            player = player.append(data, ignore_index=True)
    
    elif row[eventmsgtype] == 5:
        if not player.loc[(player["name"] == row[player1_name]), "name"].empty:
            player.loc[player["name"] == row[player1_name], "times_ball_lost"] += 1
        else:
            data["person_id"] = row[player1_id]
            data["name"] = row[player1_name]
            player.loc[player["name"] == row[player1_name], "times_ball_lost"] = 1
            player = player.append(data, ignore_index=True)
            
        if not pd.isna(row[player2_name]):
            if not player.loc[(player["name"] == row[player2_name]), "name"].empty:
                player.loc[player["name"] == row[player2_name], "times_ball_stolen"] += 1
            else:
                data["person_id"] = row[player2_id]
                data["name"] = row[player2_name]
                player.loc[player["name"] == row[player2_name], "times_ball_lost"] = 1
                player = player.append(data, ignore_index=True)
    
    elif row[eventmsgtype] == 6:
        if not player.loc[(player["name"] == row[player1_name]), "name"].empty:
            player.loc[player["name"] == row[player1_name], "fouls"] += 1
        else:
            data["person_id"] = row[player1_id]
            data["name"] = row[player1_name]
            player.loc[player["name"] == row[player1_name], "fouls"] = 1
            player = player.append(data, ignore_index=True)
    
    if game_id != row[game_id_index]:
        game_id = row[game_id_index]
    
   
    if True in ((player["name"] == row[player1_name]).values.tolist()):
        if game_id != (player.loc[player["name"] == row[player1_name], "game_id"].values[0]):
            player.loc[player["name"] == row[player1_name], "games_played"] += 1
            player.loc[player["name"] == row[player1_name], "game_id"] = game_id
    if True in ((player["name"] == row[player2_name]).values.tolist()) and not pd.isna(row[player2_name]):
        if game_id != (player.loc[player["name"] == row[player2_name], "game_id"].values[0]):
            player.loc[player["name"] == row[player2_name], "games_played"] += 1
            player.loc[player["name"] == row[player2_name], "game_id"] = game_id
                    
    if count % 100000 == 0:
        print(count, end = ", ")
    count += 1
print("Finished")

starting...
0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000, 1300000, 1400000, 1500000, 1600000, 1700000, 1800000, 1900000, 2000000, 2100000, 2200000, 2300000, 2400000, 2500000, 2600000, 2700000, 2800000, 2900000, 3000000, 3100000, 3200000, 3300000, 3400000, 3500000, 3600000, 3700000, 3800000, 3900000, 4000000, 4100000, 4200000, 4300000, 4400000, 4500000, 4600000, 4700000, 4800000, 4900000, 5000000, 5100000, 5200000, 5300000, 5400000, 5500000, 5600000, 5700000, 5800000, 5900000, 6000000, 6100000, 6200000, 6300000, 6400000, 6500000, 6600000, 6700000, 6800000, 6900000, 7000000, 7100000, 7200000, 7300000, 7400000, 7500000, 7600000, 7700000, 7800000, 7900000, 8000000, 8100000, 8200000, 8300000, 8400000, 8500000, 8600000, 8700000, 8800000, 8900000, 9000000, 9100000, 9200000, 9300000, 9400000, 9500000, 9600000, 9700000, 9800000, 9900000, 10000000, 10100000, 10200000, 10300000, 10400000, 10500000, 10600000, 10700000, 10800000, Finished


In [9]:
player.head(30)

,person_id,name,field_goals_made,field_goals_ast,field_goals_missed,times_ball_lost,times_ball_stolen,fouls,game_id,games_played,field_goals_blocked,free_throws_made,free_throws_missed,rebounds
0,722.0,Corliss Williamson,3354.0,917.0,3479.0,1200.0,430.0,2110.0,22200569.0,745.0,242.0,1813.0,713.0,2970.0
1,165.0,Hakeem Olajuwon,2008.0,649.0,2046.0,769.0,454.0,933.0,22200927.0,361.0,629.0,922.0,373.0,2665.0
2,17.0,Clyde Drexler,837.0,719.0,1078.0,340.0,242.0,350.0,22100613.0,141.0,77.0,472.0,133.0,703.0
3,53.0,Mario Elie,923.0,917.0,1103.0,476.0,343.0,687.0,22200188.0,347.0,50.0,619.0,104.0,909.0
4,51.0,Mahmoud Abdul-Rauf,616.0,311.0,784.0,154.0,79.0,249.0,22200731.0,155.0,8.0,149.0,27.0,178.0
5,1074.0,Matt Maloney,753.0,818.0,1099.0,370.0,228.0,370.0,20200448.0,284.0,14.0,199.0,51.0,478.0
6,178.0,Olden Polynice,1205.0,381.0,1351.0,444.0,154.0,1080.0,22201017.0,373.0,301.0,250.0,329.0,2339.0
7,782.0,Mitch Richmond,2286.0,1074.0,2970.0,815.0,431.0,817.0,20101188.0,365.0,73.0,1552.0,235.0,1107.0
8,788.0,Kevin Willis,1897.0,365.0,2209.0,688.0,278.0,1415.0,20601229.0,529.0,249.0,888.0,281.0,3177.0
9,258.0,Brian Grant,2204.0,686.0,2353.0,851.0,357.0,1875.0,22000068.0,609.0,329.0,1188.0,342.0,4348.0


In [8]:
from pathlib import Path  
filepath = Path('Lumiere_Reasearch/new_csv.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
player.to_csv(filepath)  